# Patient Risk Modeling: 30-Day Readmission Prediction

**Objective**: Build a predictive model and generate risk scores for each patient.

This notebook trains a machine learning model to predict 30-day hospital readmission risk, handles class imbalance with SMOTE, and exports risk scores for the dashboard.

In [ ]:
# Cell 1: Imports and Setup
import pandas as pd
import numpy as np
import json
from pathlib import Path
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import (
    classification_report, roc_auc_score, confusion_matrix, 
    roc_curve, precision_recall_curve, average_precision_score
)
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')
np.random.seed(42)

# Set up paths
DATA_DIR = Path('../data/raw')
OUTPUT_DIR = Path('../data/processed')
OUTPUT_DIR.mkdir(exist_ok=True)

print("All libraries loaded successfully")

In [ ]:
# Cell 2: Load and Initial Clean
df = pd.read_csv(DATA_DIR / 'diabetic_data.csv')
print(f"Original dataset shape: {df.shape}")

# Replace '?' with NaN for proper handling
df = df.replace('?', np.nan)

# Create binary target FIRST (before any filtering)
# 1 = readmitted within 30 days, 0 = not readmitted within 30 days
df['readmitted_30day'] = (df['readmitted'] == '<30').astype(int)

print(f"\nTarget distribution:")
print(df['readmitted_30day'].value_counts())
print(f"\n30-day readmission rate: {df['readmitted_30day'].mean()*100:.2f}%")

In [ ]:
# Cell 3: Handle Duplicate Patients
# CRITICAL: Keep patient_nbr for deduplication, drop AFTER

print(f"\nUnique patients: {df['patient_nbr'].nunique():,}")
print(f"Total encounters: {len(df):,}")
print(f"Average encounters per patient: {len(df)/df['patient_nbr'].nunique():.2f}")

# Keep only the FIRST encounter for each patient to avoid data leakage
# (A patient's second visit would have information about their first readmission)
df = df.sort_values('encounter_id').drop_duplicates(subset=['patient_nbr'], keep='first')
print(f"\nAfter keeping first encounter per patient: {len(df):,} rows")

# Store patient IDs for later reference before dropping
patient_ids = df['patient_nbr'].values.copy()

# NOW we can drop patient identifiers (after deduplication)
df = df.drop(columns=['encounter_id', 'patient_nbr'])

In [ ]:
# Cell 4: Feature Engineering
print("=" * 60)
print("FEATURE ENGINEERING")
print("=" * 60)

# 4a: Handle columns with too many missing values (>40%)
high_missing_cols = ['weight', 'payer_code', 'medical_specialty']
print(f"\nDropping high-missing columns: {high_missing_cols}")
df = df.drop(columns=high_missing_cols)

# 4b: Encode age ranges to numeric midpoints
age_mapping = {
    '[0-10)': 5, '[10-20)': 15, '[20-30)': 25, '[30-40)': 35,
    '[40-50)': 45, '[50-60)': 55, '[60-70)': 65, '[70-80)': 75,
    '[80-90)': 85, '[90-100)': 95
}
df['age_numeric'] = df['age'].map(age_mapping)
print(f"Encoded age to numeric (n={df['age_numeric'].notna().sum():,})")

# 4c: Create summary features
df['total_visits'] = df['number_outpatient'] + df['number_emergency'] + df['number_inpatient']
df['medication_intensity'] = df['num_medications'] / (df['time_in_hospital'] + 1)
print("Created derived features: total_visits, medication_intensity")

# 4d: Medication change indicator (any medication changed during stay)
med_cols = ['metformin', 'repaglinide', 'nateglinide', 'chlorpropamide', 
            'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 
            'tolbutamide', 'pioglitazone', 'rosiglitazone', 'acarbose', 
            'miglitol', 'troglitazone', 'tolazamide', 'insulin', 
            'glyburide-metformin', 'glipizide-metformin']

def count_med_changes(row):
    changes = 0
    for col in med_cols:
        if col in row.index and row[col] in ['Up', 'Down']:
            changes += 1
    return changes

df['num_med_changes'] = df.apply(count_med_changes, axis=1)
print(f"Created num_med_changes feature")

# 4e: A1C result indicator
df['A1Cresult_abnormal'] = df['A1Cresult'].apply(
    lambda x: 1 if x in ['>7', '>8'] else 0
)
print("Created A1Cresult_abnormal feature")

In [ ]:
# Cell 5: Select and Encode Features
print("\n" + "=" * 60)
print("FEATURE SELECTION AND ENCODING")
print("=" * 60)

# Define feature groups
numeric_features = [
    'time_in_hospital', 'num_lab_procedures', 'num_procedures',
    'num_medications', 'number_outpatient', 'number_emergency',
    'number_inpatient', 'number_diagnoses', 'age_numeric',
    'total_visits', 'medication_intensity', 'num_med_changes'
]

categorical_features = [
    'race', 'gender', 'admission_type_id', 'discharge_disposition_id',
    'admission_source_id', 'diabetesMed', 'change', 'A1Cresult_abnormal'
]

# Keep only selected features plus target
keep_cols = numeric_features + categorical_features + ['readmitted_30day']
df_model = df[keep_cols].copy()

print(f"Selected {len(numeric_features)} numeric features")
print(f"Selected {len(categorical_features)} categorical features")

# Handle any remaining missing values in numeric features
for col in numeric_features:
    if df_model[col].isnull().any():
        median_val = df_model[col].median()
        df_model[col] = df_model[col].fillna(median_val)
        print(f"  Filled {col} missing with median: {median_val:.2f}")

# Handle missing in categorical features
for col in categorical_features:
    if df_model[col].isnull().any():
        df_model[col] = df_model[col].fillna('Unknown')
        print(f"  Filled {col} missing with 'Unknown'")

# One-hot encode categorical features
df_encoded = pd.get_dummies(df_model, columns=categorical_features, drop_first=True)
print(f"\nAfter encoding: {df_encoded.shape[1]} total features")

# Prepare X and y
feature_cols = [col for col in df_encoded.columns if col != 'readmitted_30day']
X = df_encoded[feature_cols]
y = df_encoded['readmitted_30day']

print(f"\nFinal X shape: {X.shape}")
print(f"Final y shape: {y.shape}")
print(f"Class balance: {y.value_counts().to_dict()}")

In [ ]:
# Cell 6: Train-Test Split with Class Imbalance Handling
print("\n" + "=" * 60)
print("HANDLING CLASS IMBALANCE WITH SMOTE")
print("=" * 60)

# Split BEFORE applying SMOTE (to avoid data leakage)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training set: {X_train.shape[0]:,} samples")
print(f"Test set: {X_test.shape[0]:,} samples")
print(f"\nBefore SMOTE - Training class distribution:")
print(y_train.value_counts())

# Apply SMOTE to training data only
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

print(f"\nAfter SMOTE - Training class distribution:")
print(pd.Series(y_train_balanced).value_counts())
print(f"\nBalanced training set size: {X_train_balanced.shape[0]:,} samples")

In [ ]:
# Cell 7: Model Training and Evaluation
print("\n" + "=" * 60)
print("MODEL TRAINING")
print("=" * 60)

# Scale features for Logistic Regression
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_balanced)
X_test_scaled = scaler.transform(X_test)

# Train Logistic Regression (interpretable, good for healthcare)
print("\nTraining Logistic Regression...")
lr_model = LogisticRegression(max_iter=1000, random_state=42, C=0.1)
lr_model.fit(X_train_scaled, y_train_balanced)

# Predictions
y_pred = lr_model.predict(X_test_scaled)
y_pred_proba = lr_model.predict_proba(X_test_scaled)[:, 1]

# Evaluation
print("\n" + "=" * 60)
print("MODEL EVALUATION")
print("=" * 60)

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

roc_auc = roc_auc_score(y_test, y_pred_proba)
avg_precision = average_precision_score(y_test, y_pred_proba)
print(f"\nROC-AUC Score: {roc_auc:.4f}")
print(f"Average Precision Score: {avg_precision:.4f}")

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print(f"\nConfusion Matrix:")
print(cm)

In [ ]:
# Cell 8: Visualize Model Performance
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# ROC Curve
ax1 = axes[0]
fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
ax1.plot(fpr, tpr, color='#3498db', lw=2, label=f'ROC curve (AUC = {roc_auc:.3f})')
ax1.plot([0, 1], [0, 1], color='gray', lw=1, linestyle='--')
ax1.set_xlim([0.0, 1.0])
ax1.set_ylim([0.0, 1.05])
ax1.set_xlabel('False Positive Rate')
ax1.set_ylabel('True Positive Rate')
ax1.set_title('ROC Curve', fontweight='bold')
ax1.legend(loc='lower right')

# Precision-Recall Curve
ax2 = axes[1]
precision, recall, _ = precision_recall_curve(y_test, y_pred_proba)
ax2.plot(recall, precision, color='#e74c3c', lw=2, label=f'AP = {avg_precision:.3f}')
ax2.set_xlabel('Recall')
ax2.set_ylabel('Precision')
ax2.set_title('Precision-Recall Curve', fontweight='bold')
ax2.legend(loc='lower left')

# Confusion Matrix Heatmap
ax3 = axes[2]
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax3,
            xticklabels=['Not Readmitted', 'Readmitted'],
            yticklabels=['Not Readmitted', 'Readmitted'])
ax3.set_xlabel('Predicted')
ax3.set_ylabel('Actual')
ax3.set_title('Confusion Matrix', fontweight='bold')

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'model_performance.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Cell 9: Feature Importance Analysis
print("\n" + "=" * 60)
print("FEATURE IMPORTANCE")
print("=" * 60)

# Get coefficients from logistic regression
feature_importance = pd.DataFrame({
    'feature': feature_cols,
    'coefficient': lr_model.coef_[0]
}).sort_values('coefficient', ascending=False)

print("\nTop 10 RISK FACTORS (increase readmission risk):")
print(feature_importance.head(10).to_string(index=False))

print("\nTop 10 PROTECTIVE FACTORS (decrease readmission risk):")
print(feature_importance.tail(10).to_string(index=False))

# Visualize top features
top_features = pd.concat([
    feature_importance.head(10),
    feature_importance.tail(10)
]).sort_values('coefficient')

plt.figure(figsize=(12, 10))
colors = ['#e74c3c' if x > 0 else '#27ae60' for x in top_features['coefficient']]
plt.barh(top_features['feature'], top_features['coefficient'], color=colors)
plt.xlabel('Coefficient (Impact on Readmission Risk)')
plt.title('Top Risk & Protective Factors for 30-Day Readmission', fontweight='bold')
plt.axvline(0, color='black', linewidth=0.5)
plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'feature_importance.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Cell 10: Generate Risk Scores and Export JSON
print("\n" + "=" * 60)
print("GENERATING RISK SCORES AND EXPORTING DATA")
print("=" * 60)

# Score ALL patients (using unbalanced original data)
X_all_scaled = scaler.transform(X)
all_risk_scores = lr_model.predict_proba(X_all_scaled)[:, 1] * 100  # Convert to percentage

# Add risk scores to original dataframe
df_scored = df_model.copy()
df_scored['risk_score'] = all_risk_scores

# Calculate estimated cost per readmission
AVG_READMISSION_COST = 15000
df_scored['estimated_cost'] = (df_scored['risk_score'] / 100) * AVG_READMISSION_COST

# Add patient ID for reference
df_scored['patient_id'] = range(1, len(df_scored) + 1)

# Select top 1000 highest risk patients for dashboard (performance optimization)
df_high_risk = df_scored.nlargest(1000, 'risk_score')

print(f"Total patients scored: {len(df_scored):,}")
print(f"High-risk patients exported: {len(df_high_risk):,}")
print(f"Risk score range: {df_scored['risk_score'].min():.1f}% - {df_scored['risk_score'].max():.1f}%")
print(f"Total estimated cost exposure: ${df_high_risk['estimated_cost'].sum():,.0f}")

# Prepare export data with clean column names
export_columns = [
    'patient_id', 'age_numeric', 'time_in_hospital', 'num_medications',
    'number_diagnoses', 'number_inpatient', 'number_emergency',
    'total_visits', 'num_med_changes', 'risk_score', 'estimated_cost',
    'readmitted_30day'
]

# Rename age_numeric to age for cleaner export
export_df = df_high_risk[export_columns].copy()
export_df = export_df.rename(columns={'age_numeric': 'age'})

# Round numeric values for cleaner JSON
export_df['risk_score'] = export_df['risk_score'].round(2)
export_df['estimated_cost'] = export_df['estimated_cost'].round(2)
export_df['total_visits'] = export_df['total_visits'].astype(int)
export_df['medication_intensity'] = (df_high_risk['medication_intensity']).round(2)

# Convert to list of dictionaries for JSON
patient_risks = export_df.to_dict('records')

# Export to JSON
with open(OUTPUT_DIR / 'patient_risks.json', 'w') as f:
    json.dump(patient_risks, f, indent=2)

print(f"\nExported patient_risks.json ({len(patient_risks)} records)")

In [ ]:
# Cell 11: Generate Summary Statistics for Dashboard
print("\n" + "=" * 60)
print("GENERATING SUMMARY STATISTICS")
print("=" * 60)

# Risk distribution by bins
bins = [0, 20, 40, 60, 80, 100]
labels = ['0-20%', '20-40%', '40-60%', '60-80%', '80-100%']
df_scored['risk_bin'] = pd.cut(df_scored['risk_score'], bins=bins, labels=labels)
risk_distribution = df_scored['risk_bin'].value_counts().sort_index().to_dict()

# Age group analysis
age_bins = [0, 30, 50, 70, 100]
age_labels = ['Under 30', '30-49', '50-69', '70+']
df_scored['age_group'] = pd.cut(df_scored['age_numeric'], bins=age_bins, labels=age_labels)
age_risk = df_scored.groupby('age_group', observed=True)['risk_score'].mean().to_dict()

# Summary object
risk_summary = {
    'total_patients': int(len(df_scored)),
    'high_risk_count': int(len(df_high_risk)),
    'total_cost_exposure': float(df_high_risk['estimated_cost'].sum()),
    'avg_risk_score': float(df_high_risk['risk_score'].mean()),
    'median_risk_score': float(df_high_risk['risk_score'].median()),
    'risk_distribution': {k: int(v) for k, v in risk_distribution.items()},
    'avg_risk_by_age': {k: round(float(v), 2) for k, v in age_risk.items()},
    'model_auc': float(roc_auc),
    'readmission_rate_overall': float(df_scored['readmitted_30day'].mean() * 100)
}

# Export summary
with open(OUTPUT_DIR / 'risk_summary.json', 'w') as f:
    json.dump(risk_summary, f, indent=2)

print("Summary Statistics:")
for key, value in risk_summary.items():
    if isinstance(value, dict):
        print(f"  {key}:")
        for k, v in value.items():
            print(f"    {k}: {v}")
    elif isinstance(value, float):
        print(f"  {key}: {value:,.2f}")
    else:
        print(f"  {key}: {value:,}")

print(f"\nExported risk_summary.json")

In [ ]:
# Cell 12: Model Summary and Next Steps
print("\n" + "=" * 60)
print("MODEL SUMMARY")
print("=" * 60)

summary = f"""
MODEL: Logistic Regression with SMOTE oversampling

PERFORMANCE METRICS:
- ROC-AUC: {roc_auc:.4f}
- Average Precision: {avg_precision:.4f}

TOP RISK FACTORS:
1. {feature_importance.head(1)['feature'].values[0]}
2. {feature_importance.head(2).tail(1)['feature'].values[0]}
3. {feature_importance.head(3).tail(1)['feature'].values[0]}

EXPORTED FILES:
- patient_risks.json: {len(patient_risks)} high-risk patient records
- risk_summary.json: Dashboard summary statistics
- model_performance.png: ROC, PR curves, confusion matrix
- feature_importance.png: Top risk/protective factors

NEXT STEPS:
1. Run 03_hospital_analytics.ipynb for geographic analysis
2. Copy JSON files to dashboard/lib/ directory
3. Build Next.js dashboard
"""
print(summary)